In [ ]:
"""
bow:단어의 순서 고려 x, 단어의 빈도수를 기반으로 만든 행렬
1)단어 추출 -> 단어에 대해 정수 인덱스 설정
2)단어의 등장 횟수를 저장한 벡터를 생성
"""

In [1]:
text="정부가 .발표하는. 물가상승률과 .소비자가 느끼는 .물가상승률은 .다르다."

In [2]:
#1) text에서 마침표 제거

In [12]:
import re
token = re.sub('\.','',text)
token

'정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다'

In [13]:
from konlpy.tag import Okt

In [15]:
okt = Okt()

In [16]:
token = okt.morphs(token) # 형태소 단위 tokenize
token

['정부', '가', '발표', '하는', '물가상승률', '과', '소비자', '가', '느끼는', '물가상승률', '은', '다르다']

In [17]:
word2index = {}  # Bow 지정할 딕셔너리, {정부 : 0, 가 :1 / 단어 : 인덱스 번호}
bow = [] #{1,1 단어빈도수,단어빈도수..}

In [19]:
for v in token:
    if v not in word2index.keys(): # 중복 단어는 indexing 하지 않음 
        word2index[v] = len(word2index)
        bow.insert(len(word2index)-1,1) # 딕셔너리 특정위치에 삽입하는 insert 
        # insert(0,1) >> 0번쨰에 1을 너어라 
    else:
        # 중복 단어면 index는 그대로, 단어빈도수만 1 증가
        idx = word2index.get(v) # get : key값만 리턴
        bow[idx] = bow[idx]+1
print(word2index)
print(bow)
        

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
[2, 3, 2, 2, 3, 2, 2, 1, 1, 1]


In [37]:
# 전체문서 (n개 Doc) = [d1,d2, ... ,dn]
# 단어집합 (m개 word) = {t1,t2, ... ,tm }

### DicVectorizer : BoW 생성 

In [32]:
# Bow 만드는 class
from sklearn.feature_extraction import DictVectorizer 

In [34]:
v = DictVectorizer(sparse=False) # True : 텍스트데이터 압축해서 bow 생성 (메모리 효율)
d = [{'a':1,'b':2}, {'b':3,'c':1}] # [문서1, 문서2] {단어1:빈도수, 단어2:빈도수}
d

[{'a': 1, 'b': 2}, {'b': 3, 'c': 1}]

In [41]:
print(v.fit_transform(d)) # bow 생성 (a,b,c로 구성)
print(v.feature_names_)

v.transform({'c':5, 'd':2})

[[1. 2. 0.]
 [0. 3. 1.]]
['a', 'b', 'c']


array([[0., 0., 5.]])

### CountVectorizer
- Doc를 Token 변환 후 BoW 벡터 생성

In [47]:
# 1) 문서 -> 토큰 변환 2) 토큰의 빈도수 3) bow 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer

In [59]:
corpus=['This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?']

In [60]:
v = CountVectorizer()
v.fit(corpus)
v.vocabulary_ # {word : index 변환}

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [61]:
v.transform(corpus).toarray()
# this is the first documnet
# ARRAY 속 list는 각각의 doc

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

In [62]:
v.transform(corpus).toarray().sum(axis=0)

array([1, 4, 2, 3, 1, 1, 2, 5, 1, 3], dtype=int64)

In [64]:
v = CountVectorizer(min_df=2, max_df=4) # 단어들이 몇개의 문서에서 등장하는지
v.fit(corpus)
v.vocabulary_, v.stop_words_

({'this': 3, 'is': 2, 'first': 1, 'document': 0},
 {'and', 'last', 'one', 'second', 'the', 'third'})

In [65]:
# 두 단어가 1개의 단어로 사용
# ngram_range=(n,m) 의미? 최소 n개 최대m개의 단어를 묶어서 1개의 단어로 취급
v= CountVectorizer(ngram_range=(2,2)) # 최소:2, 최대:2 => n값이 2인 gram = 2gram (bigram)
v.fit(corpus)
v.vocabulary_

{'this is': 12,
 'is the': 2,
 'the first': 7,
 'first document': 1,
 'the second': 9,
 'second second': 6,
 'second document': 5,
 'and the': 0,
 'the third': 10,
 'third one': 11,
 'is this': 3,
 'this the': 13,
 'the last': 8,
 'last document': 4}

In [66]:
v = CountVectorizer(stop_words=['and','is','the','this'])
v.fit(corpus)
v.vocabulary_ # stop words로 빠진 나머지가 vocabulary로 들어옴

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [67]:
# ngram_range(1,2)
v = CountVectorizer(token_pattern ='t\w+') # t로 시작하는것만 bow로
# \w+ 단어 단위
# \w 문자 단위
v.fit(corpus)
v.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [68]:
# ngram_range(1,2) t로 시작하는것중 최소1 최대 2
v = CountVectorizer(ngram_range=(1,2),token_pattern='t\w+') # t로시작하는 
v.fit(corpus)
v.vocabulary_ 

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

In [69]:
v = CountVectorizer(analyzer='char') # 문자단위로 생성
v.fit(corpus)
v.vocabulary_ 

{'t': 16,
 'h': 8,
 'i': 9,
 's': 15,
 ' ': 0,
 'e': 6,
 'f': 7,
 'r': 14,
 'd': 5,
 'o': 13,
 'c': 4,
 'u': 17,
 'm': 11,
 'n': 12,
 '.': 1,
 'a': 3,
 '?': 2,
 'l': 10}

In [75]:
v = CountVectorizer(stop_words='english')
v.fit(corpus)
v.vocabulary_ # stop words로빠진나머지가 vocabulary로들어옴

{'document': 0, 'second': 1}

In [76]:
v=CountVectorizer(stop_words=['and','is','the','this'])
v.fit(corpus)
v.vocabulary_

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [79]:
v=CountVectorizer()
v.fit(corpus)
v.vocabulary_

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

- trasnform() : Corpus로 fitting 된 BoW를 바탕으올 Array 생성

In [81]:
# 새로운문서 입력 -> BoW로변환
'This is the second document'
v.transform(['This is the second document']) # sparse matrix
v.transform(['This is the second document']).toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]], dtype=int64)